In [ ]:
!pip install pyspark

transfofrmations practice

In [2]:
from pyspark.sql.functions import col,desc,upper,lower,sum,when
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("test").getOrCreate()

In [3]:
df = spark.read.option("header",True).option("inferschema",True).csv("/content/sample_data/california_housing_test.csv")

when otherwise // case when in sql

In [ ]:
# when ,otherwise
df.withColumn("new_col",when(col("longitude") >= -112,"around california").\
              when(col("longitude") < -112,"away from california").otherwise("I dont know where you are")).show()


In [15]:
df2 = df.withColumn("test",when(col("longitude") >= -119,"around california")\
              .otherwise("I dont know where you are"))

In [ ]:
# pviot : aggregation function
df2.groupBy(col("longitude")).pivot("test").sum("latitude").show()

In [ ]:
# changing name of a column
df.withColumnRenamed("longitude","long").show()


In [ ]:
# changing data type
# using cast()

df.withColumn("longitude",col("longitude").cast("float")).printSchema()
# OR
df.select(col("longitude").cast("float")).printSchema()

sort and orderby sre same in Pyspark dataframe

but in sql sort by and orderby are diff.

sortby sorts within partitons

In [ ]:

# sorting
df.sort(col("longitude").desc()).show()

df.orderBy(col("longitude").desc()).show()
# orderBy
df.select(col("longitude"),col("housing_median_age")).orderBy(col("housing_median_age").desc()).show()

In [ ]:
# # like
# df.where(col("longitude").like("-1%")).show()
# df.filter(col("longitude").like("-1%")).show()
df.filter(col("latitude").like("3%")).select("longitude").show()

In [ ]:
# count number of records
df.count()

In [ ]:
# distinct
df2 = df.dropDuplicates(["longitude"])

In [ ]:
df2.show()

In [ ]:
l=["sort and orderby sre same", "in Pyspark dataframe but in sql",\
  " sort by and orderby are diff. sortby sorts within partitons"]

In [ ]:
r = spark.sparkContext.parallelize(l,5)

In [ ]:
r.glom().collect()

In [ ]:
rf = r.filter(lambda x:x!=" ")
rf.glom().collect()

In [ ]:
rep=rf.repartition(3)

In [ ]:
rep.glom().collect()

In [ ]:
# reduceByKey
df.select(col("longitude")).rdd

In [ ]:
# fill and fillna
df3 = spark.createDataFrame([["hello","jk"],["world","lk"],["from","sk"],[None,"sd"],["moon",None]],schema="name string,pet_name string")
df3.show()

In [ ]:
df3.na.fill({"name":"not given","pet_name":"NA"}).show()

In [ ]:
# sample
df3.sample(0.6).show()

In [55]:
# UDF : user defined function

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType,IntegerType
def total_pay(a,b)->int:
  return int(a+b)
totalPay = udf(lambda x,y : total_pay(x,y),IntegerType())
df.select(totalPay(col("longitude"),col("latitude")).alias("location")).show()

In [ ]:
conf = df.rdd.context.getConf()
# df.rdd.context.appName
conf.set("spark.ui.showConsoleProgress",False)
conf.getAll()